In [3]:
from simple_salesforce import Salesforce as sf
login_sf = sf(username = 'svc_rao@groupon.com', password = 'Password-1Je8IR8frn0gkbS68Eq4OHZmM', security_token = '')

import pandas as pd
from simple_salesforce import bulk
from megatron import saving as m
import pygsheets
import os

from connect import sf_query


In [4]:
gc = pygsheets.authorize(service_file=os.path.expanduser("~")+"\\.megatron\\google_service_account_secret.json")

sh = gc.open_by_key('1-XYkQmKeUKZRCuiLrzPuzqVYKttSbLW587U901rK734')

# define the following worksheets
live_accounts = sh.worksheet_by_title("live_accounts")
db_output = sh.worksheet_by_title("db_output")
live_acc = live_accounts.get_values("B2","B25000")

acc_list = []
db_ids = []

for acc in live_acc:
    for x in acc:
        acc_list.append(x)


In [5]:
for acc in acc_list:
    try:
        output = sf_query("SELECT Id,DNBoptimizer__DnBCompanyRecord__c FROM Account WHERE Id",acc)
    except:
        pass
    
    try:    
        db_id = output['records'][0]['DNBoptimizer__DnBCompanyRecord__c']
        db_ids.append(db_id)
    except:
        db_ids.append('None')


In [6]:
db_output = []

for db_id in db_ids:
    if db_id == None:
        pass
    else:
        output = sf_query("SELECT Id,DNBoptimizer__BusinessName__c,DNBoptimizer__DUNSNumber__c,DNBoptimizer__GlobalUltimateDUNSNumber__c,DNBoptimizer__DelistedIndicator__c,DNBoptimizer__EmployeeCountTotal__c,DNBoptimizer__LocationType__c,DNBoptimizer__NAICSDescription1__c,DNBoptimizer__NAICSDescription2__c,DNBoptimizer__NAICSDescription3__c,DNBoptimizer__OutofBusinessIndicator__c,DNBoptimizer__PrimaryCityName__c,DNBoptimizer__PrimaryCountryCode_ISO__c,DNBoptimizer__PrimaryCountryName__c,DNBoptimizer__PrimaryPostalCode__c,DNBoptimizer__PrimaryStreetAddress__c,DNBoptimizer__TelephoneNumber__c,DNBoptimizer__WebAddress__c,DNBoptimizer__YearStarted__c FROM DNBoptimizer__DnBCompanyRecord__c where Id",db_id)
        db_output.append(output['records'][0])


In [7]:
db_output = pd.DataFrame(db_output)
db_output = db_output.iloc[:,1:]
db_output = db_output.rename(columns={'Id':'db_id'})


In [8]:

db_map = {
    'account_id': acc_list,
    'db_id': db_ids
    }

db_map = pd.DataFrame(db_map)


In [9]:
db_table = db_map.join(db_output.set_index('db_id'), on='db_id',how='left')


In [10]:
m.df_to_gdoc(db_table,'1-XYkQmKeUKZRCuiLrzPuzqVYKttSbLW587U901rK734','db_output')

print("Success")

[21:02:09] [OK] saving dataframe to https://docs.google.com/spreadsheets/d/1-XYkQmKeUKZRCuiLrzPuzqVYKttSbLW587U901rK734/edit (sheet: db_output, how: replace, shape: (25522, 20)) succeeded
Success
